In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(subscription_id="04bf6e4a-98b1-41c0-b2b3-c7362661b532", name="azure-ml-udacity-workspace", resource_group="azure-ml-udacity")
exp = Experiment(workspace=ws, name="udacity-project-experiment")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_cluster_name = "azureml-compute1"
try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V3', max_nodes=1)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": uniform(0.5, 2.0),
    "max_iter": choice(100, 200, 500)
})

# Specify a Policy
termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory="./", 
    compute_target=compute_cluster,
    entry_script="./train.py",
    pip_requirements_file_path="./requirements.txt")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                           hyperparameter_sampling=ps,
                           policy=termination_policy,
                           primary_metric_name='Accuracy',
                           primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                           max_total_runs=10,
                           max_concurrent_runs=1)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails

hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

joblib.dump(best_run, "hyperdrive_model.pkl")

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(data_url)

In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=20,
    task="classification",
    primary_metric="accuracy",
    training_data=x.join(y),
    label_column_name="y",
    n_cross_validations=3)

In [ ]:
# Submit your automl run

automl_exp = Experiment(ws, "automl-project-udacity-experiment")
automl_run = automl_exp.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

automl_best_run = automl_run.get_best_child()
automl_best_run_metrics = automl_best_run.get_metrics()

print('Best Run Id: ', automl_best_run.id)
print('\n Accuracy:', automl_best_run_metrics['accuracy'])

joblib.dump(automl_best_run, "automl_model.pkl")